In [1]:
import gammapy
print(gammapy.__version__)

import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore, Observation
from gammapy.datasets import MapDataset
from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.makers import MapDatasetMaker

1.0


In [2]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")
obs_id = [23523] # just one observation 
obs1 = data_store.get_observations(obs_id)[0]

In [3]:
crab_pos = SkyCoord(184.557, -5.784, unit='deg', frame='galactic') 
obs_pos=obs1.pointing_radec
ebins = np.geomspace(0.5,100,100) # a fine binning in true energy
energy_axis = MapAxis.from_edges(
    ebins, unit="TeV", name="energy", interp="log"  
)
energy_axis_true = MapAxis.from_edges(
    ebins, unit="TeV", name="energy_true", interp="log"  
)
migra_axis = MapAxis.from_bounds(
    0.2, 5, nbin=160, node_type="edges", name="migra"
)
geom = WcsGeom.create(
    skydir=obs_pos,
    binsz=0.02,
    width=(3.5, 3.5),
    frame="icrs",
    proj="CAR",
    axes=[energy_axis],
)

In [4]:
%%time
maker = MapDatasetMaker(selection=['exposure'])
reference = MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, migra_axis=migra_axis)

dataset = maker.run(reference, obs1)

CPU times: user 1.14 s, sys: 238 ms, total: 1.37 s
Wall time: 1.4 s


In [19]:
def make_exposure_factors(livetime, aeff, pointing, coords):
    """Get energy dispersion for a given event and true energy axis.

        Parameters
        ----------
        livetime : `~astropy.units.quantity.Quantity`
            livetime of the observation    
        aeff : `~gammapy.irf.effective_area.EffectiveAreaTable2D`
            effective area from the observaton
        pointing : `~astropy.coordinates.SkyCoord`
            Pointing position of the observation. Should be a single coordinates.
        coords : `~gammapy.maps.coord.MapCoord`
            coordinates on which the model will be evaluated. Needs true energy axis and skycoord.

        Returns
        -------
        exposure : `~numpy.ndarray`
            the exposure values for the unbinned evaluator.
        """
    offsets = coords.skycoord.separation(pointing)
    exposure = aeff.evaluate(offset=offsets, energy_true=coords["energy_true"])
    return (exposure * livetime).to("m2 s")

In [37]:
coords = dataset.exposure.geom.get_coord(sparse=True) # sparse is important for speed
# we should make sure that we also give sparse coords

In [38]:
exp_manual = make_exposure_factors(obs1.observation_live_time_duration, obs1.aeff, obs1.pointing_radec, coords)

In [39]:
# some timing
%timeit make_exposure_factors(obs1.fixed_pointing_info.duration, obs1.aeff, obs1.pointing_radec, coords)
print(f"for {np.prod(exp_manual.shape)} values")

371 ms ± 424 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
for 3031875 values


In [40]:
np.all(exp_manual == dataset.exposure.quantity)

True